In [1]:
import numpy as np
import copy
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d
from scipy.spatial.transform import Rotation
np.set_printoptions(threshold=np.infty)
from utils_data_elab import VIC_OFF_X, VIC_OFF_Y, get_scenes, scene, obstacle, run, get_obs_info, get_scene_index


from postproc_utils import list_dataset, \
    dict_dataset, \
    load_dataset, \
    open_json, \
    load_obstacles, \
    apply_rototranslation, \
    plot_scene, \
    plot_2dscene, \
    arena_filter, \
    obstacles_filter, \
    obstacles_filter2, \
    obstacles_filter3, \
    quad_dist2, \
    inside_walls, \
    keep_borders, \
    generate_abs_bounding_obstacle, \
    generate_abs_bounding_obstacles, \
    get_point_labels, \
    compute_distv1, \
    stack_labels_dists, \
    get_labels

mpl.rcParams['xtick.labelsize'] = 25
mpl.rcParams['ytick.labelsize'] = 25
# set axis label size to 25
mpl.rcParams['font.size'] = 25

path_to_json = "./data/"


# Load data

In [20]:
model = "Transformer_v17_lr2"
run = "49"

run_pred_fn = "./data/full_runs_prediction/"+model+"/full_run_pred_labels_run_"+run+".npy"
run_labels_fn = "./data/full_runs_prediction/"+model+"/full_run_labels_labels_run_"+run+".npy"

run_labels = np.load(run_labels_fn)
run_labels = np.squeeze(run_labels)         # removing the last column

run_pred = np.load(run_pred_fn)
run_pred = np.argmax(run_pred, axis=-1)     # the predictions are actually probabilities --> argmax

labelled_mmw = json.load(open(path_to_json+"/run_"+run+"/labelled_mmw_run_"+run+".json"))["labelled_mmw"]
labelled_mmw_ = []
for i in range(len(labelled_mmw)):
    for j in range(len(labelled_mmw[i])//200):
        labelled_mmw_.append(labelled_mmw[i][j*200:(j+1)*200])
labelled_mmw = labelled_mmw_

print(" Shapes: ", run_labels.shape, run_pred.shape, len(labelled_mmw))

 Shapes:  (1343, 12, 200) (1343, 12, 200) 1355


In [40]:
def unroll(data):
    ret = [x for x in data[0]]
    for i in range(1, len(data)):
        ret.append(data[i][-1])
    return np.array(ret)

unrolled_labels = unroll(run_labels)
pos = np.stack(labelled_mmw[:len(unrolled_labels)])[:, :, :3]

unrolled_labels = np.stack([unrolled_labels, pos[:, :, 0], pos[:, :, 1], pos[:, :, 2]], axis=-1)


(1354, 200, 4)


### Stack data

In [16]:
# compute accuracy
# accuracy = np.mean(run_labels == run_pred)
# print("Accuracy: ", accuracy)
labels = np.hstack(np.vstack(run_labels))
preds = np.hstack(np.vstack(run_pred))

accuracy = np.mean(labels == preds)
print("Accuracy: ", accuracy)

Accuracy:  0.7619052494415488
